Nightly Validation Pipeline Errors for night of {{ params.day_obs }}
=====

In [ ]:
day_obs = "2024-06-18"
instrument = "LSSTComCamSim"

In [ ]:
if instrument == "LATISS":
    butler_alias = "/repo/embargo"
else:
    butler_alias = "embargo_or4"

day_obs_int = int(day_obs.replace('-',''))

In [ ]:
import lsst.daf.butler as dafButler
from lsst.daf.butler import DimensionNameError
from lsst.pipe.base import Pipeline
import tabulate
from dataclasses import dataclass

@dataclass
class error_summary:
    visit: int
    detector: int
    error_messages: list

b = dafButler.Butler(butler_alias, collections=f"{instrument}/nightlyValidation")

In [ ]:
log_visit_detector = set([(x.dataId['exposure'], x.dataId['detector']) for x in b.registry.queryDatasets("isr_log", where=f"day_obs = {day_obs_int}")])
print("Number of ISR records in butler: {:d}".format(len(log_visit_detector)))

isr_visit_detector = set([(x.dataId['exposure'], x.dataId['detector']) for x in b.registry.queryDatasets("postISRCCD", where=f"day_obs = {day_obs_int}")])
print("Number of successful ISR results: {:d}".format(len(isr_visit_detector)))

pvi_visit_detector = set([(x.dataId['visit'], x.dataId['detector']) for x in b.registry.queryDatasets("calexp", where=f"day_obs = {day_obs_int}")])
print("Number of successful processCcd results: {:d}".format(len(pvi_visit_detector)))

missing_pvis = set(log_visit_detector - pvi_visit_detector)
missing_visits = [x[0] for x in missing_pvis]
print("Number of unsuccessful processCcd attempts (no resulting calexp): {:d}".format(len(missing_pvis)))


In [ ]:
def make_error_summaries(log_dataset_types_exposure, log_dataset_types_visit, data_ids):
    error_summaries = []
    for visit, detector in data_ids:
    
        visit_errors = []
        
        for ds_types in log_dataset_types_exposure:
            log_messages = b.get(ds_types, dataId={"instrument": instrument, "exposure": visit, "detector": detector})
            isr_errors = [msg for msg in log_messages if msg.levelno > 30]
            visit_errors.extend(isr_errors)
        
        for ds_types in log_dataset_types_visit:
            try:
                log_messages = b.get(ds_types, dataId={"instrument": instrument, "visit": visit, "detector": detector})
            except DimensionNameError: # Visit records can be missing due to corrupted headers.
                errors = []
            else:
                errors = [msg for msg in log_messages if msg.levelno > 30]
            finally:
                visit_errors.extend(errors)
    
        error_summaries.append(error_summary(visit=visit, detector=detector, error_messages=visit_errors))
    return error_summaries

In [ ]:
def make_url_from_visit(visit):
    s = str(visit)
    day_string = f"{s[0:4]}-{s[4:6]}-{s[6:8]}"
    counter = int(s[8:])
    if instrument == "LATISS":
        instrument_url = "auxtel"
    else:
        instrument_url = "comcam_sim"
    #url = f"https://usdf-rsp.slac.stanford.edu/rubintv/summit/auxtel/monitor/event/{day_string:s}/{counter:d}"
    # Example: https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/comcam_sim/event?key=comcam_sim/2024-06-21/calexp_mosaic/000011/comcam_sim_calexp_mosaic_2024-06-21_000011.png
    url = f"https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/{instrument_url:s}/event?key={instrument_url:s}/{date_string:s}/calexp_mosaic/{counter:06d}/comcam_sim_calexp_mosaic_{date_string:s}_{counter:06d}.png"
    
    return url

In [ ]:
pipeline = Pipeline.from_uri(f"$DRP_PIPE_DIR/pipelines/{instrument}/nightly-validation.yaml#step1")
pipeline_graph = pipeline.to_graph(registry=b.registry)

In [ ]:
last_records = log_visit_detector
table_contents = []
last_task = "isr"
for task in pipeline_graph.tasks:
    if task in ("isr", "getRegionTimeFromVisit"):
        continue
    records = set([(x.dataId['visit'], x.dataId['detector']) for x in b.registry.queryDatasets(task+"_log")])
    error_summaries = make_error_summaries(["isr_log"], [last_task+"_log"], set(last_records) - set(records))
    for e in error_summaries: 
        msg = e.error_messages[0].message[:1000] if e.error_messages else f"No {task}; last {last_task}"
        table_contents.append((e.visit, e.detector, "<a href=\"" + make_url_from_visit(e.visit) + "\" target=\"_blank\">img</a>", msg)) 
        
    print(f"  {len(records):d} {task} records")
    last_task = task
    last_records = records

In [ ]:
table = tabulate.tabulate(table_contents, tablefmt='unsafehtml', headers=("Visit", "Det", "Img", "Error Message"))
table